In [1]:
from typing import List, Tuple, Dict, Optional
from collections import defaultdict

from vllm import LLM, SamplingParams


In [17]:
"""
Two models do not fit on 4 V100s, so you should run this script and paste the output into vllm_PV_final.ipynb.
"""

# V100 settings.
llm = LLM(model="deepseek-ai/deepseek-math-7b-instruct",
          max_num_batched_tokens=8192,
          trust_remote_code=True,
          dtype="float16",
          tensor_parallel_size=4)


WARNING 09-30 01:37:55 config.py:1651] Casting torch.bfloat16 to torch.float16.
INFO 09-30 01:37:55 config.py:890] Defaulting to use mp for distributed inference
INFO 09-30 01:37:55 llm_engine.py:213] Initializing an LLM engine (v0.6.0) with config: model='deepseek-ai/deepseek-math-7b-instruct', speculative_config=None, tokenizer='deepseek-ai/deepseek-math-7b-instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=4, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False,

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


(VllmWorkerProcess pid=7541) INFO 09-30 01:37:56 multiproc_worker_utils.py:215] Worker ready; awaiting tasks


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


(VllmWorkerProcess pid=7544) INFO 09-30 01:37:56 multiproc_worker_utils.py:215] Worker ready; awaiting tasks
(VllmWorkerProcess pid=7549) INFO 09-30 01:37:56 multiproc_worker_utils.py:215] Worker ready; awaiting tasks
(VllmWorkerProcess pid=7544) (VllmWorkerProcess pid=7541) ERROR 09-30 01:37:56 multiproc_worker_utils.py:226] Exception in worker VllmWorkerProcess while processing method init_device: Cannot re-initialize CUDA in forked subprocess. To use CUDA with multiprocessing, you must use the 'spawn' start method, Traceback (most recent call last):
ERROR 09-30 01:37:56 multiproc_worker_utils.py:226] Exception in worker VllmWorkerProcess while processing method init_device: Cannot re-initialize CUDA in forked subprocess. To use CUDA with multiprocessing, you must use the 'spawn' start method, Traceback (most recent call last):
(VllmWorkerProcess pid=7541) (VllmWorkerProcess pid=7544) (VllmWorkerProcess pid=7549) ERROR 09-30 01:37:56 multiproc_worker_utils.py:226]   File "/opt/conda/

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


RuntimeError: Cannot re-initialize CUDA in forked subprocess. To use CUDA with multiprocessing, you must use the 'spawn' start method

In [18]:

sampling_params = SamplingParams(
    max_tokens=4096,
    temperature=1.0,
    top_k=1,
    top_p=1.0,
)



In [19]:
lean_game_dict = {
    'header': 'import Mathlib\nimport Aesop\n\nset_option maxHeartbeats 0\n\nopen BigOperators Real Nat Topology Rat\n\n',
    'problem': '/-- The second and fourth terms of a geometric sequence are $2$ and $6$. Which of the following is a possible first term?\nShow that it is $\\frac{2\\sqrt{3}}{3}$.-/\ntheorem amc12b_2003_p6 (a r : \u211d) (u : \u2115 \u2192 \u211d) (h\u2080 : \u2200 k, u k = a * r ^ k) (h\u2081 : u 1 = 2)\n    (h\u2082 : u 3 = 6) : u 0 = 2 / Real.sqrt 3 \u2228 u 0 = -(2 / Real.sqrt 3) := by\n',
    'old_code' : '  simp_all only [h\u2080, Nat.cast_one, Nat.cast_zero, Nat.cast_succ, Nat.cast_zero]\n  have h\u2083 : a * r = 2 := by linarith\n  have h\u2084 : a * r ^ 3 = 6 := by linarith\n  have h\u2085 : r ^ 2 = 3 := by\n    nlinarith\n',
    'tactic_state' : 'a r : ℝ\nu : ℕ → ℝ\nh₀ : ∀ (k : ℕ), u k = a * r ^ k\nh₁ : a * r ^ 1 = 2\nh₂ : a * r ^ 3 = 6\nh₃ : a * r = 2\nh₄ : a * r ^ 3 = 6\nh₅ : r ^ 2 = 3\n⊢ a * r ^ 0 = 2 / √3 ∨ a * r ^ 0 = -(2 / √3)\n'
}


In [20]:
output_1 = r"""Summarize the proof state:

We have a geometric sequence defined by \( u_k = a \cdot r^k \). We know that the second term \( u_1 = 2 \) and the fourth term \( u_3 = 6 \). We have derived the following equations:
1. \( a \cdot r = 2 \)
2. \( a \cdot r^3 = 6 \)
3. \( r^2 = 3 \)

Our goal is to show that the first term \( u_0 \) is either \( \frac{2}{\sqrt{3}} \) or \( -\frac{2}{\sqrt{3}} \).

Discussion:

The proof state is consistent with the problem statement and the given conditions. We have derived the necessary equations to relate the terms of the geometric sequence. The next steps would involve solving for \( a \) and \( r \) using the derived equations and then verifying the possible values for \( u_0 \).

The proof is on the right track as it uses the given conditions to derive useful equations. However, the proof is incomplete as it does not yet solve for \( a \) and \( r \) explicitly.
"""

In [21]:
def policy_value_suggest_comments(lean_game_dict: Dict, discussion_context : str, num: int = 5) -> str:
    # We should call the LLM now.

    res = discussion_context + f"""Here is the tactic state at this point:
```lean4
"""
    res += lean_game_dict['tactic_state']
    res += f"""
```
Please suggest 5 ideas to complete the proof.
Please delimit each idea with <IDEA></IDEA> tags.
Rate the likelihood that this proof will succeed on a scale of 1 (very unlikely) to 10 (very likely).
Please delimit this rating with <RATING></RATING> tags.

<IDEA>"""
    return res

In [22]:
prompt_2 = policy_value_suggest_comments(lean_game_dict, output_1)
print(prompt_2)

outputs = llm.generate(
    prompt_2,
    sampling_params=sampling_params
)
outputs = outputs[0].outputs[0].text + "\n"
print(outputs)


Summarize the proof state:

We have a geometric sequence defined by \( u_k = a \cdot r^k \). We know that the second term \( u_1 = 2 \) and the fourth term \( u_3 = 6 \). We have derived the following equations:
1. \( a \cdot r = 2 \)
2. \( a \cdot r^3 = 6 \)
3. \( r^2 = 3 \)

Our goal is to show that the first term \( u_0 \) is either \( \frac{2}{\sqrt{3}} \) or \( -\frac{2}{\sqrt{3}} \).

Discussion:

The proof state is consistent with the problem statement and the given conditions. We have derived the necessary equations to relate the terms of the geometric sequence. The next steps would involve solving for \( a \) and \( r \) using the derived equations and then verifying the possible values for \( u_0 \).

The proof is on the right track as it uses the given conditions to derive useful equations. However, the proof is incomplete as it does not yet solve for \( a \) and \( r \) explicitly.
Here is the tactic state at this point:
```lean4
a r : ℝ
u : ℕ → ℝ
h₀ : ∀ (k : ℕ), u k = a * 

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.59s/it, est. speed input: 301.45 toks/s, output: 100.06 toks/s]

1. Solve the system of equations to find the values of a and r.</IDEA>
<IDEA>2. Substitute the values of a and r into the equation for u_0.</IDEA>
<IDEA>3. Simplify the equation for u_0 to show that it equals either 2/√3 or -2/√3.</IDEA>
<IDEA>4. Use the properties of geometric sequences to verify that the values of u_0 are correct.</IDEA>
<IDEA>5. Use the derived equations to show that the values of u_0 are indeed 2/√3 or -2/√3.</IDEA>

<RATING>8</RATING>

